# Curation Estaciones Meteorologicas

Estudiar La cantidad de datos validos disponibles para cada variable en cada estacion y obtener el numero de estaciones con los datos minimos necesarios para el estudio

In [1]:
setwd("~/Repositories/AirQualityCOVID")

In [2]:
source("src/general.R")

## Funciones para filtrar datos

In [3]:
suppressMessages(library(lubridate))
suppressMessages(library(tidyverse))
suppressMessages(library(plyr))

In [4]:
get.info <- function(st) {
    
    dataframe <- read.csv(paste("data/Curation/AEMET/Values/",
                                st,
                                ".csv", sep=""))
    
    dataframe$fecha <- ymd(dataframe$fecha)
    period <- as.numeric(ymd("2020-12-31")-ymd("2013-01-01")) +2
    
    cbind(data.frame(site=st,
                     start_dt=min(dataframe$fecha),
                     end_dt=max(dataframe$fecha),
                     miss_dy=period - nrow(dataframe)
                    ),
          rbind(apply(dataframe, 2, 
                      function(df, period) {
                          (sum(!is.na(df)) / period)
                      }, period))
         )
}

In [8]:
sitesAQ <- read.csv("data/Curation/sitesAQ.csv",
                    stringsAsFactor=TRUE)

all.data <- do.call(rbind.fill, lapply(levels(sitesAQ$site), get.info))
all.data <- all.data[, -which("sol" == names(all.data))]

In [9]:
head(all.data)

,site,start_dt,end_dt,miss_dy,fecha,indicativo,nombre,provincia,altitud,tmed,⋯,horatmax,dir,velmedia,racha,horaracha,presMax,horaPresMax,presMin,horaPresMin,siteAQ
,<fct>,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,es0041a,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.0000000,0.9996579,0.9993158,0.9996579,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000
2,es0110a,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.0000000,0.9996579,0.9993158,0.9996579,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000
3,es0115a,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,0.9993158,⋯,1.0000000,0.9418406,0.9483407,0.9418406,1.0000000,0.9846049,0.9846049,0.9846049,0.9842627,1.0000000
4,es0118a,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,0.9993158,⋯,1.0000000,0.9418406,0.9483407,0.9418406,1.0000000,0.9846049,0.9846049,0.9846049,0.9842627,1.0000000
5,es0120a,2013-01-01,2020-12-31,106,0.9637359,0.9637359,0.9637359,0.9637359,0.9637359,0.8925761,⋯,0.9637359,0.4830653,0.4950393,0.4830653,0.9637359,NA,NA,NA,NA,0.9637359
6,es0817a,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,0.9904208,⋯,1.0000000,0.9811837,0.9883681,0.9811837,1.0000000,0.9890523,0.9890523,0.9890523,0.9890523,1.0000000


# Guardar Datos

```R
write.csv(all.data,
          "data/Curation/AEMET/info_sites.csv",
          row.names=F)
```

In [17]:
info <- data.frame()
percent <- c(0.5, 0.7, 0.8, 0.9)

for (pr in percent) {
    info <- rbind(info,
                  rbind(apply(all.data[,(10:ncol(all.data))], 2, 
                              function(df){
                                  sum(df > pr, na.rm=TRUE)
                              }))
                 )
}

names(info) <- names(all.data)[10:ncol(all.data)]
row.names(info) <- percent

In [18]:
info

,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,presMax,horaPresMax,presMin,horaPresMin,siteAQ
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0.5,66,66,66,61,66,61,53,54,53,59,55,55,55,55,66
0.7,66,66,66,61,66,61,53,53,53,59,55,55,55,55,66
0.8,65,66,65,61,65,61,53,53,53,59,55,55,55,55,66
0.9,58,63,58,60,58,60,50,50,50,58,52,52,52,52,65
